In [3]:
from KalshiClientsBaseV2ApiKey import ExchangeClient
import time
import json
import uuid
import pandas as pd
import numpy as np
import re

from cryptography.hazmat.primitives import serialization
from cryptography.hazmat.backends import default_backend

import requests
from datetime import datetime
import pytz
import warnings
warnings.filterwarnings('ignore')

In [4]:
# 加载密钥
def load_private_key_from_file(file_path):
    with open(file_path, "rb") as key_file:
        private_key = serialization.load_pem_private_key(
            key_file.read(),
            password=None,  # or provide a password if your key is encrypted
            backend=default_backend()
        )
    return private_key

In [5]:
prod_key_id = "e79af1da-54e5-46a3-9269-c2de1dc81de0" 
prod_private_key = load_private_key_from_file('../kalshi-key.key')
prod_api_base = "https://api.elections.kalshi.com/trade-api/v2"
exchange_client = ExchangeClient(exchange_api_base=prod_api_base, key_id = prod_key_id, private_key = prod_private_key)
exchange_client.get_exchange_status()

msg_string 1732238058018GET/trade-api/v2/exchange/status
signature EY45L5ANEcBh545Obf1DQyCmLjf1CO6JwWmhiYhER+S6WAmE2xp1cBKIULI7QWQDkJYSlAQt0g4fLNJGqwpwvvE8nH+z8ZTLoUiDyHtYHLFb+VzEEGoffZA826WovcUz1z5bOA5d6tb9hw8GEu/0G8V3Hp3JRJlcVPQxd29EDaRQ5QdTlW9h2BRqDqE2h8vct7W/rddt21E8iHsSmOCgnkc3SMHZzCOpZiLcVNgCbds37PR9zQaCxn2/CdRN8+LvW+aL8LyhPUPw/YqASNP8eRWkBbdq9QpWhHR2RXDACLKf/CinR5QgQio97V8iz0J2Yhd/eJmZMoK6lPNhmRmK3g==


{'exchange_active': True, 'trading_active': True}

# get event_ticker  series_ticker

In [6]:
events_data = pd.read_csv('../data_events.csv')

In [7]:
len(set(events_data['event_ticker'].values)) == events_data.shape[0]

True

In [8]:
event_ticker = events_data.loc[0,'event_ticker']
event_ticker  # 某一个事件的ticker

'JOBLESS-21AUG28'

In [9]:
series_ticker = events_data.loc[0,'series_ticker']
series_ticker  # 某一个类别的ticker,比如 Economics 类别

'KXJOBLESS'

# get Series

In [10]:
series = exchange_client.get_series(series_ticker=series_ticker)

msg_string 1732238059357GET/trade-api/v2/series/KXJOBLESS
signature OB5E244EYTzq0/3yRLE0coru0ucq0gR7CjuqaEUiLrxWjR1Ne/SzGCobLBHRNp+PLQLW2cozYFDKEzDLQVGQWw/dsi+idiRPMgKZvRhDsJQEmTR3yO+VYsHDQEFAXNYDVw3Tg6lY6x3oCPrU7Kjx+Dx4uM+qTWtC9q6t4jpdaCXY8qS5HAkiMC4x/SVoVGWa8vkr12SuadiNz/PddwZ3IcfS5sUWe37tgT9l3esU6sg6RI+muU1a4OJXl6JXYhAGLUzx1g9To2kIm8MfU1wPcJ7XkryFrTNxzlaQG+skfeFVwXpWZ+tIF8NiLQMgn6ZTazAiV4gqmpUCSaACZ5r4JA==


In [11]:
series

{'series': {'ticker': 'KXJOBLESS',
  'frequency': 'weekly',
  'title': 'Initial jobless claims',
  'category': 'Economics',
  'tags': ['Employment'],
  'settlement_sources': [{'url': 'https://www.dol.gov/ui/data.pdf',
    'name': 'Department of Labor'}],
  'contract_url': 'https://kalshi-public-docs.s3.us-east-1.amazonaws.com/regulatory/product-certifications/JOBS.pdf'}}

# get_market

In [12]:
market = exchange_client.get_market(ticker='JOBLESS-21AUG28-C350')   # market ticker
market

msg_string 1732238060317GET/trade-api/v2/markets/JOBLESS-21AUG28-C350
signature olhsoV3XR3/TE2tA434VmkM2ehhsKFfSjO186rJghGA05/XQRclCrCKdnFLYSCDJ9uLDtudj7SA4snuwq3b/1rfahfS3ooNLpUgX5tZr4ddV/ssLfk0z8qZgvtaS0huEfXQbLw2udEet+1lGupatRcIGvUf3oPMLgwjmMOTv8HxuTIqaQLwZ9+WngAFsBBqx4jjKyfDgW+/rPX02RJqKleRtj5nGMfDrhfGHEfeliIEoG69u+rYPTae+ybkdHaoKJYDVUlnCnayEGMXEWkwlfFVkUneLiB5UF4/kDylT8wxYO/UPXkkO48e3byMharrtLRAPre8bw0IUT4TaATNIzQ==


{'market': {'ticker': 'JOBLESS-21AUG28-C350',
  'event_ticker': 'JOBLESS-21AUG28',
  'market_type': 'binary',
  'title': 'Will initial jobless claims be higher than 350,000 for the week ending August 28, 2021?',
  'subtitle': '>350K',
  'yes_sub_title': 'Above 350,000',
  'no_sub_title': '350,000 or below',
  'open_time': '2021-08-26T20:30:00Z',
  'close_time': '2021-09-01T23:00:00Z',
  'expected_expiration_time': '2033-08-26T20:19:00Z',
  'expiration_time': '2021-09-02T22:00:00Z',
  'latest_expiration_time': '2021-09-02T22:00:00Z',
  'settlement_timer_seconds': 0,
  'status': 'finalized',
  'response_price_units': 'usd_cent',
  'notional_value': 100,
  'tick_size': 1,
  'yes_bid': 35,
  'yes_ask': 38,
  'no_bid': 62,
  'no_ask': 65,
  'last_price': 36,
  'previous_yes_bid': 35,
  'previous_yes_ask': 38,
  'previous_price': 36,
  'volume': 2698,
  'volume_24h': 0,
  'liquidity': 0,
  'open_interest': 1771,
  'result': 'no',
  'settlement_value': 0,
  'can_close_early': False,
  'expira

# get_markets

In [13]:
event_ticker
# JOBLESS-21AUG28

'JOBLESS-21AUG28'

In [14]:
exchange_client.get_markets(event_ticker=event_ticker)['markets']

msg_string 1732238061415GET/trade-api/v2/markets
signature ZDhqXJD49tCbdCDO1QOqG6gBVEdvijz95YWggSSQ9yPPzzU/77f8hH4fPiMue5/kSTIACldopeoDvUsPKfJ8+rxmfDNcBb/LPCZ11Kg5OtHaVi4ppYHFtq0oQ3iV0mthwsgdwgQK2AcUnB4SEuWYXsKIaT6hS2tBx/lJd1CZ512+vqe87FTYEcl2qQpz385t9g84HM9o1Y85q2mIcwmGingKOY5LIXDc2J/6DTE44f+NwtFjCM5eXUvMYB6ReOs5VY+/YfOPDjY4CXAkXW6B5jyhw/2kLQ+vaiLH3kvsWsL9snREL+Cd/30GepHgPDUDvBiTXqmwBlig3Z4lxfBS0w==


[{'ticker': 'JOBLESS-21AUG28-C350',
  'event_ticker': 'JOBLESS-21AUG28',
  'market_type': 'binary',
  'title': 'Will initial jobless claims be higher than 350,000 for the week ending August 28, 2021?',
  'subtitle': '>350K',
  'yes_sub_title': 'Above 350,000',
  'no_sub_title': '350,000 or below',
  'open_time': '2021-08-26T20:30:00Z',
  'close_time': '2021-09-01T23:00:00Z',
  'expected_expiration_time': '2033-08-26T20:19:00Z',
  'expiration_time': '2021-09-02T22:00:00Z',
  'latest_expiration_time': '2021-09-02T22:00:00Z',
  'settlement_timer_seconds': 0,
  'status': 'finalized',
  'response_price_units': 'usd_cent',
  'notional_value': 100,
  'tick_size': 1,
  'yes_bid': 35,
  'yes_ask': 38,
  'no_bid': 62,
  'no_ask': 65,
  'last_price': 36,
  'previous_yes_bid': 35,
  'previous_yes_ask': 38,
  'previous_price': 36,
  'volume': 2698,
  'volume_24h': 0,
  'liquidity': 0,
  'open_interest': 1771,
  'result': 'no',
  'settlement_value': 0,
  'can_close_early': False,
  'expiration_value

In [15]:
len(exchange_client.get_markets(event_ticker=event_ticker)['markets'])

msg_string 1732238062351GET/trade-api/v2/markets
signature q7y2I2ZFibttYo9tvumV7YInEqBBfjTbB3Law4W5nXQiEDd8vcLibIQW8v4SAvnnbgbEKxeAAZ3qLki4X+3NiKvn0UYM8M7R2cHt9JbBAAiCjXJ56KwsYUlWao28SHEwBlH6EJbWvsJXDxLh3NhSP1D3I5ehAU/T1uLmxkl2LZ6rJdohG4nqi3nkjBklvohuHbOJMHkvBsaENlQE6v1PHHtSlM8hXeictecSEbfqOpjSGTxKbuc5a6WSSo6l5MqoqWldiyKcwp57uWu73Q775RxDH7zWPk1nAyW6RLn456sgEpxvPTe2nDboUEr00TeOt4uZ9b8vYmLzd58I/dTl0A==


1

In [16]:
# data_markets = pd.read_csv('../data_markets.csv')

# # cursor = 0
# while cursor != '':
#     try:
#         markets, cursor = exchange_client.get_markets(cursor=cursor)['markets'], exchange_client.get_markets(cursor=cursor)['cursor']
#     except Exception as e:
#         print(e)
#         for _ in range(10):
#             markets, cursor = exchange_client.get_markets(cursor=cursor)['markets'], exchange_client.get_markets(cursor=cursor)['cursor']
#             if markets != []:
#                 print('connection success!')
#                 break
#     finally:
#         data_markets = pd.concat([data_markets,pd.DataFrame(markets)])

# 很耗时

In [17]:
# data_markets.shape

In [18]:
# cursor

In [19]:
# data_markets.to_csv('../data_markets.csv',index=False)

In [20]:
data_markets = pd.read_csv('../data_markets.csv')

In [21]:
data_markets[data_markets['event_ticker']=='KXRECESSAPPT-25']

,ticker,event_ticker,market_type,title,subtitle,yes_sub_title,no_sub_title,open_time,close_time,expected_expiration_time,...,category,risk_limit_cents,rules_primary,rules_secondary,strike_type,custom_strike,floor_strike,cap_strike,settlement_value,fee_waiver_expiration_time
0,KXRECESSAPPT-25-TG,KXRECESSAPPT-25,binary,Will Tulsi Gabbard be appointed to the Trump A...,NaN,Tulsi Gabbard,Tulsi Gabbard,2024-11-20T15:00:00Z,2025-08-01T14:00:00Z,2025-08-01T14:00:00Z,...,Politics,0,If Tulsi Gabbard is appointed to the Trump Adm...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,KXRECESSAPPT-25-RFK,KXRECESSAPPT-25,binary,Will Robert F. Kennedy Jr. be appointed to the...,NaN,Robert F. Kennedy Jr.,Robert F. Kennedy Jr.,2024-11-20T15:00:00Z,2025-08-01T14:00:00Z,2025-08-01T14:00:00Z,...,Politics,0,If Robert F. Kennedy Jr. is appointed to the T...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,KXRECESSAPPT-25-PH,KXRECESSAPPT-25,binary,Will Pete Hegseth be appointed to the Trump Ad...,NaN,Pete Hegseth,Pete Hegseth,2024-11-20T15:00:00Z,2025-08-01T14:00:00Z,2025-08-01T14:00:00Z,...,Politics,0,If Pete Hegseth is appointed to the Trump Admi...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,KXRECESSAPPT-25-MG,KXRECESSAPPT-25,binary,Will Matt Gaetz be appointed to the Trump Admi...,NaN,Matt Gaetz,Matt Gaetz,2024-11-20T15:00:00Z,2025-08-01T14:00:00Z,2025-08-01T14:00:00Z,...,Politics,0,If Matt Gaetz is appointed to the Trump Admini...,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# get_market_history

In [22]:
data_markets.loc[0,'ticker']

'KXRECESSAPPT-25-TG'

In [23]:
# param = {
#     'ticker':'KXRECESSAPPT-25-TG',
#     'limit':100
# }

# for i in range(data_markets.shape[0]):
#     try:
#         market_resp = exchange_client.get_market_history(ticker=data_markets.loc[i,'ticker'])
#         break
#     except Exception as e:
#         print(e)

# market_resp = exchange_client.get_market_history(ticker=data_markets.loc[0,'ticker'])
market_resp = exchange_client.get_market_history(ticker='KXRECESSAPPT-25-TG')

msg_string 1732238067150GET/trade-api/v2/markets/KXRECESSAPPT-25-TG/history
signature Pqp+0aemQeE6O+8AHrVj+4g4/VfAJVORhAKF3D2cqOsUXSwBpI901CJO6sF640oLgsFbRxMYpltJd0niJVRulds0XK8tZcRTXRG8qc+s4SMtwCuVgtJAxA2/1T9x7Iu/k82If+769xfl9a17NArvllN3pYT11IkdWjlOIyxSGoITSULhIv7DhtbveJirwhM+VH78D5aM8JAFnBBVE9hfkA5Po0sGY/QSe+X6QUX0hSJHx8oufC2IUZ74KlnzEkO1IVeiOPlkbTstneytbgWwgUQV0jEI/zHhvjN0/CNyPdBE+FNij/ctO5T8UERYjiQ2l62NqkR+FJ2VoSXydjIo0g==


HttpError: HttpError(404 Not Found)

In [24]:
with open('../kalshi-key.txt','r',encoding='utf-8') as file:
    kalshi_key = file.read()
kalshi_key = re.sub('\n','',kalshi_key)
kalshi_key

'MIIEogIBAAKCAQEAwI/tSTxdRXfAHehenHMXhO3khd17ih1Db2KRQE8Vl5ZEbsdIG5Q/Kn8J0f6Ig8jo+EM/i3d3CBYyiLPQB/zTn+jhRpmyt9y1qCgIPfi7iMphVL/57wm9/S2q4K40c/hXT5pBA56W7bPxeVLIrmy/NSkPnQXG13TEJ54CuJyrI6Oc63glMaTyCTjHVBdUJcvVyHL2RZfbaUhRpurfYJJUzaqsqugSK7/ozuowXHZiHghNAAsbtygiA0CFGxAMJKOJsDtADjmEyYVCjHpLMJerCm28RUlg5SuckGx3uT8IGzaNo0znF79NPy26+lBs7MUzuYwo1XhGZbuigRt/FNPMbQIDAQABAoIBAEmHa+M4MqFxHaIN9fqllAiFUmqvbRoLHJHuwGofCK6uiYkHfYhSbmBKzDQsjnJMCHGsNBqMTp5+42MBzmEOcLyyDzuHOxAsLg1849DOJsYQCWR0PfrB7JM9me9bddoGrUhPLZ41M4qm2IEsVEcObWDeQgECkK/6/V/nXUCjSlI73//ILVc146+PdoG48j20GyTq+SOzAFsyMClUuo3uqTMuCv5Kb7ZrsrY1P5sePtZ/fjGIeg7BN5xOnDCr2/uRZL1ywLVct5HnlH/liMxHXftPqh/f4n0ekxlFLNwVOcTRdOGalBmDjPWY3gQUouBmM4xwtuYvmM5/nIfvq0IgNYECgYEA96K7oICI93vEF5xoOlKLKWU79KoWPEQ7zA7dEdvJSgDyYzm7JDUWhiQoFMux62wQ+SzDTQ7WAQv0/nyCZMtZm41ST+nnbCmpM+iMNnP5wRTnS5f+gpKcSNO+WMgUp180YGG8cnMYXU09X+KTSAxkXTeHpfu1eRe8g/F+tAJK6c0CgYEAxxD7ezcQ1u2K4mI4SFAX27CU1+duJMOS99MSEd4EdJEhBHrT+K+PgLPc9nVXyxKsy+bvbR1BJ2KKq0zotmHWR45wsht88GziFBFGklb+HKv

In [25]:
# ISO 8601 格式的日期时间字符串
# iso_date = "2021-07-26T21:00:00Z"
iso_date = '2021-08-01T23:00:00Z'

# 将 ISO 8601 格式的字符串转换为 datetime 对象
dt_object = datetime.strptime(iso_date, "%Y-%m-%dT%H:%M:%SZ")

# 将 datetime 对象转换为 Unix 时间戳
timestamp = int(time.mktime(dt_object.timetuple()))
timestamp

1627830000

In [ ]:
base_url = "https://api.elections.kalshi.com/trade-api/v2"
endpoint = "series/{series_ticker}/markets/{ticker}/candlesticks"

series_ticker = "KXMTA"  
ticker = "MTA-21AUG01-A2.23" 

url = f"{base_url}/{endpoint}".replace("{series_ticker}", series_ticker).replace("{ticker}", ticker)

params = {
    'start_ts': 1627304400,  # 开始时间
    'end_ts': 1627830000,    # 结束时间
    'period_interval': 1440                       # 时间间隔，例如 1 天
}
headers = {
    "accept": "application/json",
    "Authorization": 'Bearer '+ prod_key_id
}
response = requests.get(url, headers=headers)

response.text

'{"error":{"code":"missing_parameters","message":"missing parameters","service":""}}'

# 获取 Bearer Token

In [ ]:
import uuid
import kalshi_python
from kalshi_python.models import *
from pprint import pprint

config = kalshi_python.Configuration()
# Comment the line below to use production
config.host = "https://api.elections.kalshi.com/trade-api/v2"

# Create an API configuration passing your credentials.
# Use this if you want the kalshi_python sdk to manage the authentication for you.
kalshi_api = kalshi_python.ApiInstance(
    email='YOUR_EMAIL_HERE',
    password='YOUR_PASSWORD_HERE',
    configuration=config,
)


In [ ]:

# Alternatively you can manage the authentication yourself by doing:
# kalshi_api = kalshi_python.ApiInstance(
#     configuration=config,
# )
# loginResponse = kalshi_api.login(LoginRequest(email='YOUR_EMAIL_HERE', password='YOUR_PASSWORD_HERE'))
# token = loginResponse.token
# kalshi_api.set_api_token(loginResponse.token)
# # The token used above can eventually expire if you stop using it for more than 30 minutes.
# # In that case you should redo the process by doing a new login request and setting the api token again in the same way.


In [ ]:

# Checks if the exchange is available.
exchangeStatus = kalshi_api.get_exchange_status()
print('Exchange status response: ')
pprint(exchangeStatus)